# Last Minute Moneyball: Maximize Your Impact Before Election Day
## Author: Eric Bolton

**Note: This notebook and analysis were first written in the aftermath of the 2018 midterm elections, as reflected in the accompanying text. However, the latest outputs reflect data from the 2020 general election.**

Many reasons exist to exercise one's right to vote. Beyond simply fulfilling one's civic duty, voting constitutes a way to make one's voice heard, to feel part of something bigger, and to participate in the great democratic experiment. One might want to vote simply because increased turnout [affects policy outcomes](http://www.nyu.edu/gsas/dept/politics/seminars/hajnal_s06.pdf).

Yet, every election cycle in America, tens of millions of eligible voters stay at home. It's easy to see why: in elections where hundreds of thousands vote - sometimes millions - the perception may be that one's voice is likely to be lost in the cacophony of the crowd, providing little motivation to head to the polls. One might argue that one's vote only truly "counts" if their vote is the one to push their candidate of choice over the brink, an event that many see as far too unlikely to warrant a visit to the voting booth.

But this perception may be misguided. Consider this: if each vote has only a one in ten million chance of changing the outcome of an election, but that election affects billions of dollars in spending at a national scale, then each vote is effectively "worth" hundreds of dollars. In this light, voting is a rational decision per se, if only as an act of charity. This is the argument made by renowned statistician [Andrew Gelman](https://80000hours.org/2016/11/why-the-hour-you-spend-voting-is-the-most-socially-impactful-of-all/).

So, using this reasoning, how much is one's vote truly worth? This project seeks to answer that in the context of the 2018 Midterm Elections for the House and Senate.

# Simulating Elections

In order to come up with an answer, we first need to discover the likelihood that one's vote would have changed the outcome of individual races. Leading up to Election Day, this perceived likelihood depended on two key factors: what the polls were saying about each race (closer races, and more uncertain races have higher odds of being decided by one vote) and what kind of turnout is expected (races with fewer voters have higher odds of being decided by one vote). To this end, I designed a `Country` object that would read and organize the relevant data\* into `State` objects, which in turn were associated with `Race` objects.

\*Available in the [Github repository](https://github.com/edbltn/voter-power)

In [1]:
import elections

# Create national election predictor
country = elections.Country('../data')

First I looked at the polls for each race leading up to Election Day. I only considered polls that were taken during October or November\*. Here I assumed that races with no such polls available were essentially uncontested - an assumption that held true in a vast majority of cases, but that definitely weakens the analysis. Follow up work should seek to use voting history for counties as a predictor for races where polling data was lacking.

My next assumption was that the number of polling respondents favoring a given candidate would follow a Binomial distribution $B(n, p)$ where $n$ is the number of polling samples and $p$ is the probability of any voter casting a vote for the given candidate. I approximated this distribution as a Normal distribution with mean $n\hat{p}$ and variance $n\hat{p}(1-\hat{p})$ where $\hat{p}$ was the observed proportion of polling respondents favoring the given candidate.

I used a truncated distribution (`scipy.stats.truncnorm`) to ensure that it would only include values between 0 and $n$. Dividing by $n$, this yielded a distribution for $p$, the probability that any voter would pick the given candidate, based on polling data:

$
p\sim N_0^{1}\left(\hat{p},\sqrt{\frac{\hat{p}(1-\hat{p})}{n}}\right)
$

Where $N_0^1$ designates a truncated normal distribution between 0 and 1. We'll use $N_0^1$ as a shorthand for the above expression going forward.

Let $V$ be the number of votes cast for the given candidate. Now, for a given turnout $t$, the probability that exactly $k = \left\lceil\frac{t}{2}\right\rceil$ voters would pick the given candidate (hence - all votes are "decisive," assuming a two-candidate election), would be:

$
P[V = k]_{p\sim N_0^1} = \left({t \atop k}\right)p^k(1-p)^{t-k}
$

Thus the expected tipping point probability can be computed by integrating over $N_0^1$.

$
\mathbb{E}[P[V=k]]_{p\sim N_0^1} = \int_{0}^1 \left({t \atop k}\right)p^k(1-p)^{t-k}N_0^1(p)dp
$

This can be computed using `scipy.stats.rv_continuous.expect`.

Now, the only missing factor is the expected turnout. Here, I cheated a little and used the actual turnout on Election Day; future work should use projected turnout based on early vote counts and voting history.

\*Polls were obtained from [Real Clear Politics](https://www.realclearpolitics.com/elections/2018/)

In [2]:
district_probabilities = {}
for state in country.states:
    if state.postal_code == 'DC':
        continue
    for district in state.districts.values():
        p = district.tipping_point_probability()
        district_probabilities[district.code] = p

/Users/ericbolton/voter-power/env/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2626: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  vals = integrate.quad(fun, lb, ub, **kwds)[0] / invfac


Typically, only one Senate seat is contested per state. But the exits of Al Franken (D-MN) and Thad Cochran (R-MS) from the Senate required special elections in Minnesota and Mississippi. This meant that these states each had two Senate elections in 2018. For simplicity's sake, I'm picking the larger of the two probabilities in these cases.

(*Note that the seats are examined in alphabetical order of their codes - i.e. 'MS' will be seen before 'MS2'*)

In [3]:
senate_seat_probabilities = {}
for state in country.states:
    for seat in state.senate_seats.values():
        p = seat.tipping_point_probability()
        if not seat.code.endswith('2'):
            senate_seat_probabilities[state.postal_code] = p
            senate_seat_probabilities[state.name] = p                        
        else:
            senate_seat_probabilities[seat.code] = p

In [4]:
ec_probabilities = {}
for state in country.states:
    for electors in state.electoral_college.values():
        p = electors.tipping_point_probability()
        ec_probabilities[electors.code] = p

# Simulating Chamber Votes

Now that we know the likelihood of one's vote changing the outcome for each individual race, we want to compute the likelihood of each individual race changing the outcome of key House and Senate votes. To predict the outcome of votes, I used partisanship scores available on [VoteView](https://voteview.com), which tracks the partisanship of each senator and representative based on their voting history. For candidates whose partisanship scores weren't available, I simply used the average score of elected officials from their party in their state.

I created a `ChamberVote` object that looks at how Senators or Representatives actually voted on a past issue\* in order to predict, using logistic regression, how a new class of elected officials would vote. In this example, I use the Senate vote to repeal Obamacare, which narrowly failed on July 28, 2017.

\*This data is also available via [VoteView](https://voteview.com).

In [6]:
from votes import ChamberVote

vote = ChamberVote('../data/votes/obamacare_senate.csv', 
                   country.official_scorer)

Now, all that remains to be done is to simulate different elections and use that to estimate how often the Obamacare vote would come down to just one vote in either chamber. In the past, polling has been known to exhibit a national bias of up to 2\% toward one party or the other, so I am including a bias factor for each election cycle.

In [7]:
def get_tipping_points(ec, ec_diff):
    sorted_ec = (
        ec
        .set_index(['party', 'code'])
        .sort_values(by=['party', 'margin'])
        .groupby('party')
        .cumsum()[['value']]
        .rename(columns={'value': 'reverse_total'})
        .join(ec.set_index(['party', 'code'])[['value']])
        .join(
            ec
            .set_index(['party', 'code'])
            .sort_values(by=['party', 'margin'], ascending=False)
            .groupby('party')
            .cumsum()[['value']]
            .rename(columns={'value': 'ec_total'})
        )
    )
    sorted_ec['ec_diff'] = sorted_ec['ec_total'] - (sorted_ec.value.sum()/2)
    tipping_points = (
        sorted_ec[(sorted_ec.value > sorted_ec.ec_diff)
                  & (sorted_ec.value > sorted_ec.reverse_total - sorted_ec.value)
                  & (sorted_ec.ec_diff >= 0)]
        .reset_index()
    )

    # Return the state codes for all of the tipping points
    return tipping_points.code

In [ ]:
import math
import numpy as np
import progressbar

from collections import defaultdict

n_simulations = 1000
senate_power = defaultdict(lambda: 0)
house_power = defaultdict(lambda: 0)
ec_power = defaultdict(lambda: 0)

p = progressbar.ProgressBar(term_width = 80)

for n in p(range(n_simulations)):
    
    election_bias = np.random.normal(0, 0.01)
    
    gov = country.simulate_government(election_bias, vote)
    senate = gov[gov.chamber == 'senate']
    house = gov[gov.chamber == 'house']
    ec = gov[gov.chamber == 'ec']

    senate_diff = sum(senate.value[senate.party == 'R']) - sum(senate.value[senate.party == 'D'])
    house_diff = sum(house.value[house.party == 'R']) - sum(house.value[house.party == 'D'])
    ec_diff = sum(ec.value[ec.party == 'R']) - sum(ec.value[ec.party == 'D'])
    senate_diff += np.sign(ec_diff) # The vice president

    for code in get_tipping_points(ec, ec_diff):
        ec_power[code] += 1 / n_simulations
    for code in get_tipping_points(senate, senate_diff):
        senate_power[code] += 1 / n_simulations
    for code in get_tipping_points(house, house_diff):
        house_power[code] += 1 / n_simulations
            


 77% (772 of 1000) |################      | Elapsed Time: 0:02:15 ETA:   0:00:38

In [ ]:
import os 

state_codes_df = pd.read_csv(os.path.join('../data/state_info', 'state_codes.csv'))
state_codes = dict(zip(state_codes_df['postal_code'], state_codes_df['name']))

Finally, we can combine the probability that a voter would be decisive in their race, and the probability that their candidate of choice would be decisive in their chamber, by multiplying them (assuming they represent independent events).

$
P[\textrm{"Voter is decisive"} \cap \textrm{"Elected Official is decisive"}] = P[\textrm{"Voter is decisive"}] \cdot P[\textrm{"Elected Official is decisive"}]
$

We can then multiply this result by the cost of Obamacare (\$133 billion) estimated by the Congressional Budget Office. This will tell us how much each vote was worth in the 2018 election when it came to the issue of healthcare.

In [ ]:
from collections import defaultdict
from heapq import nlargest
from operator import itemgetter
import pandas as pd

# Federal budget for 2019
cost = 4.45e10

senate_seats = defaultdict(lambda: [])
senate_seat_values = {}
for senate_seat, p in senate_seat_probabilities.items():
    if len(senate_seat) > 2:
        continue
    senate_seats['state'].append(state_codes[senate_seat])
    senate_seats['decisive_voter_probability'].append(p * senate_power[senate_seat])
    
    value = p * cost * senate_power[senate_seat]
    senate_seat_values[state_codes[senate_seat]] = value
    value_string = f'${round(value, 2):.2f}'
    senate_seats['value_of_vote'].append(value_string)

districts = defaultdict(lambda: [])
district_values = {}
for district, p in district_probabilities.items():
    districts['district'].append(district)
    districts['decisive_voter_probability'].append(p * house_power[district])
    
    value = p * cost * house_power[district]
    district_values[district] = value
    value_string = f'${round(value, 2):.2f}'
    districts['value_of_vote'].append(value_string)
    
ec = defaultdict(lambda: [])
electors_values = {}
for electors, p in ec_probabilities.items():
    if len(electors) > 2:
        continue
    ec['district'].append(state_codes[electors])
    ec['decisive_voter_probability'].append(p * ec_power[electors]) 
    
    value = p * cost * ec_power[electors]
    electors_values[state_codes[electors]] = value
    value_string = f'${round(value, 2):.2f}'
    ec['value_of_vote'].append(value_string)
    
    
senate_seats_df = pd.DataFrame.from_dict(senate_seats)
districts_df = pd.DataFrame.from_dict(districts)
electors_df = pd.DataFrame.from_dict(ec)

display(electors_df.nlargest(10, 'decisive_voter_probability'))
display(districts_df.nlargest(10, 'decisive_voter_probability'))
display(senate_seats_df.nlargest(10, 'decisive_voter_probability'))

In [ ]:
from plot_us_map import plot_districts, plot_states

max_ec = max(electors_df.value_of_vote.apply(lambda r: float(r[1:])))
max_state = max(senate_seats_df.value_of_vote.apply(lambda r: float(r[1:])))
max_district = max(districts_df.value_of_vote.apply(lambda r: float(r[1:])))
district_values['DCAL'] = 0
plot_states(electors_values, 0, max_ec + 1)
plot_states(senate_seat_values, 0, max_state + 1)
plot_districts(district_values, 0, max_district + 1)

In [ ]:
import math
import numpy as np
import progressbar

from collections import defaultdict

sims = 10000
p = progressbar.ProgressBar(term_width = 80)
voter_power = defaultdict(lambda: 0)

for n in p(range(sims)):
    election_bias = np.random.normal(0, 0.02)
    gov = country.simulate_government(election_bias, vote)
    gov['chamber_margin'] = np.nan
    
    totals = (
        gov
        .groupby(['chamber', 'party'])
        .value.sum()
        .reset_index()
        .set_index('chamber')
    )
    margins = totals[totals.party == 'R'].value - totals[totals.party == 'D'].value
    # Add VP!
    margins.loc['senate'] += np.sign(margins.loc['ec'])
    
    for chamber in ('ec', 'house', 'senate'):
        if all(margins >= 0):
            for code in get_tipping_points(gov[gov.chamber == chamber], margins.loc[chamber]):
                voter_power[code, chamber] += 1 / sims

        elif all(margins <= 0):
            for code in get_tipping_points(gov[gov.chamber == chamber], margins.loc[chamber]):
                voter_power[code, chamber] += 1 / sims
     

Shapefiles can be found [here](https://www.census.gov/geo/maps-data/data/cbf/cbf_state.html) for the states (simply select 'Census 2000' and then select 'United States') and [here](https://koordinates.com/layer/96077-us-116th-congressional-districts/) for the districts.

It's immediately striking that votes in some states had a negligible value, while votes in others were worth thousands of dollars each. Perhaps, in a better system, each vote would weigh equally. Analyses such as these show where the difference needs to be made, perhaps by working to reduce gerrymandering, or changing voting rules to make individual votes more impactful. One example might be the effort [spearheaded by Maine's 2nd](https://thehill.com/opinion/campaign/418058-maines-2nd-district-outcome-proves-value-of-ranked-choice-voting) in this election cycle to switch to a ranked choice voting system (votes in that tightly contested race, by the way, were worth \$3,000 apiece). But beyond that, in the states and districts where voting mattered most, it is my hope that analyses such as these might cause those who stayed at home to reconsider their decision in future elections.

In [ ]:
import pandas as pd

cost = 4.45e10

rows = []
for code, p in ec_probabilities.items():
    if len(code) > 2:
        continue
    power = (voter_power[code, 'ec'] * p)
    if code == 'DC':
        rows.append({
            'state':  state_codes[code],
            'decisive_voter_probability': power,
            'value_of_vote':  f'${round(power * cost, 2):.2f}'
        })
        continue
    power += (voter_power[code, 'senate'] * senate_seat_probabilities[code]) 
    power += (voter_power[code + '2', 'senate'] * senate_seat_probabilities[code + '2']) 
    rows.append({
        'state':  state_codes[code],
        'decisive_voter_probability': power,
        'value_of_vote':  f'${round(power * cost, 2):.2f}'
    })
    
state_df = pd.DataFrame(rows)
state_df.sort_values(by='decisive_voter_probability', ascending=False).head(10)

In [ ]:
rows = []
for code, p in district_probabilities.items():
    state = code[:2]
    power = (voter_power[code, 'house'] * p)
    if state in ('NE', 'ME'):
        power += (voter_power[code.replace('E', 'ECD'), 'ec'] * ec_probabilities[state])
    power += (voter_power[state, 'ec'] * ec_probabilities[state])
    power += (voter_power[state, 'senate'] * senate_seat_probabilities[state])
    rows.append({
        'district': code,
        'decisive_voter_probability': power,
        'value_of_vote':  f'${round(power * cost, 2):.2f}'
    })
rows.append({
    'district': 'DCAL',
    'decisive_voter_probability': ec_probabilities['DC'],
    'value_of_vote': f'${round(ec_probabilities["DC"] * cost, 2):.2f}'
})
    
district_df = pd.DataFrame(rows)
district_df.sort_values(by='decisive_voter_probability', ascending=False).head(10)

Thus, voters in many states performed a substantial act of charity simply by making their way to their local polling station. When this analysis was first performed in 2018, in the closely followed and hotly contested race between Kyrsten Sinema and Martha McSally, each vote was worth almost \$8000, based on just one of many Senate votes. The races in Nevada and Indiana were worth thousands of dollars per vote as well.

When these states voted on November 8th, 2018, millions of lives, and billions of dollars hung in the balance. Yet the states only had middling turnout, at 49.10\%, 47.50\%, and 46.90\% each, compared to 50.10\% nationally.

Finally, let's see how these results look on a map:

In [ ]:
from plot_us_map import plot_districts, plot_states

max_state = max(state_df.value_of_vote.apply(lambda r: float(r[1:]))) + 1
max_district = max(district_df.value_of_vote.apply(lambda r: float(r[1:]))) + 1
plot_states(dict(zip(state_df.state, state_df.decisive_voter_probability * cost)), 0, max_state)
plot_districts(dict(zip(district_df.district, district_df.decisive_voter_probability * cost)), 0, max_district)